# Load data

In [1]:
import pandas as pd
import numpy as np
import scipy as sy
import os
from scipy import stats
from more_itertools import numeric_range
import math
import seaborn as sb

In order to align the start time of the behavioral (lever press timing) and miniscope (neural activities) recordings, we initiate the miniscope recording prior to the behavioral training and subtract the time difference.

More specifically, we determine the time difference when the behavior session begins and when the miniscope begins recording. Then, we subtract the timing in the next cell.


In [2]:
# Specify the time in sec to be deleted from analysis.  This is the time difference between the onset of the miniscope and behavioral recordings.
bf_time=[4.3]
t=0

Defining the directory.  Please specify the directory accordingly.

In [3]:
sample_dir = 'C:/Users/AK_ROG/GitGithub/Research_Projects/sucrose-neuro/Standardizing'

files = "example_neural_data.csv"

Delta f over f is one type of standardization 

The following code outputs a csv file containing the Delta F/F data

In [4]:
### Delta f over f
#     for files in os.listdir(sample_dir):
print(bf_time[t])

raw_data = pd.read_csv(files)
name = files[:-4]
before_time = bf_time[t]
data = raw_data.copy()
data = data - min(data.iloc[:,range(1,len(data.columns))].min())
data['ceil_time'] = (data['Time(ms)']/100).apply(np.ceil).astype(int)*100
data = data.groupby(['ceil_time']).mean()
data = data.reset_index()
data['Time(ms)'] = data['ceil_time']
data = data.drop('ceil_time', axis=1)
data = data.rename(columns={'Time(ms)':'Time(s)'})
data['Time(s)'] = round(data['Time(s)']/1000,1)
delete_indx = data[data['Time(s)'] <= before_time].index
#print(delete_indx)
data.drop(delete_indx, inplace=True)
data.reset_index(drop=True, inplace=True)
data['Time(s)'] = data['Time(s)'] - before_time
#data.head()
cols = data.columns
z_data = pd.DataFrame(columns=cols)
z_data['Time(s)'] = data['Time(s)']
for xc in range(1,len(cols)):
    for ran in range(0,math.ceil(len(data)/3000)):
        if (ran+1)*3000<len(data):
            baseline = data.iloc[range(ran*3000,(ran+1)*3000),xc].mean()
            z_data.iloc[range(ran*3000,(ran+1)*3000),xc] = (data.iloc[range(ran*3000,(ran+1)*3000),xc] - baseline)/baseline
        else:
            baseline = data.iloc[range(len(data)-3000,len(data)),xc].mean()
            z_data.iloc[range(ran*3000,len(data)),xc] = (data.iloc[range(ran*3000,len(data)),xc] - baseline)/baseline

z_data["Time(s)"]=round(z_data["Time(s)"],1)
z_data.to_csv(sample_dir + name +'_deltaF_over_F.csv', index=False)

4.3


Z-score is another type of standardization 
The following code outputs a csv file containing the z-score

In [5]:
print(files)
print(bf_time[t])

raw_data = pd.read_csv(files)
name = files[:-4]
before_time = bf_time[t]
data = raw_data.copy()
data['ceil_time'] = (data['Time(ms)']/100).apply(np.ceil).astype(int)*100
data = data.groupby(['ceil_time']).mean()
data = data.reset_index()
data['Time(ms)'] = data['ceil_time']
data = data.drop('ceil_time', axis=1)
data = data.rename(columns={'Time(ms)':'Time(s)'})
data['Time(s)'] = round(data['Time(s)']/1000,1)
delete_indx = data[data['Time(s)'] <= before_time].index
#print(delete_indx)
data.drop(delete_indx, inplace=True)
data.reset_index(drop=True, inplace=True)
data['Time(s)'] = data['Time(s)'] - before_time
#data.head()
cols = raw_data.columns
cols = cols[1:]
z_data = pd.DataFrame()
for col in cols:
    col_zscore = col + '_zscore'
    z_data['Time(s)'] = data['Time(s)']
    z_data[col] = (data[col] - data[col].mean())/data[col].std(ddof=0)

z_data["Time(s)"]=round(z_data["Time(s)"],1)
#z_data.tail()
z_data.to_csv(sample_dir + name +'_standardized.csv', index=False)

example_neural_data.csv
4.3


<ipython-input-5-35cab666695d>:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  z_data[col] = (data[col] - data[col].mean())/data[col].std(ddof=0)
<ipython-input-5-35cab666695d>:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  z_data[col] = (data[col] - data[col].mean())/data[col].std(ddof=0)
<ipython-input-5-35cab666695d>:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axi